# Example: Conditional Parameter Grids

This example shows the usage of `PyExperimenter` with a conditional parameter grid. We will programmatically define the parameter combinations of a support vector machine, instead of generating the entire cartesian product from the parameters defined in the config file.  

To execute this notebook you need to install:
```
pip install py_experimenter
pip install scikit-learn
```

## Experiment Configuration File
This notebook shows an example execution of `PyExperimenter` based on an experiment configuration file. Further explanation about the usage of `PyExperimenter` can be found in the [documentation](https://tornede.github.io/py_experimenter/usage.html). Here, we only define keyfields and resultfields and do not set the parameter values in the experiment configuration file as we will create the parameter grid programmatically.

In [1]:
import os

content = """
PY_EXPERIMENTER:
  n_jobs: 1

  Database:
    provider: sqlite
    database: py_experimenter
    table: 
      name: example_conditional_grid
      keyfields:
        dataset:
          type: VARCHAR(50)
        cross_validation_splits:
          type: int
        seed: 
          type: int
        kernel: 
          type: VARCHAR(50)
        gamma: 
          type: VARCHAR(50)
        degree: 
          type: VARCHAR(50)
        coef0: 
          type: VARCHAR(50)
      result_timestamps: false
      resultfields:
        train_f1: DECIMAL
        train_accuracy: DECIMAL
        test_f1: DECIMAL
        test_accuracy: DECIMAL

  CUSTOM:
    path: sample_data
"""

# Create config directory if it does not exist
if not os.path.exists('config'):
    os.mkdir('config')
    
# Create config file
experiment_configuration_file_path = os.path.join('config', 'example_conditional_grid.yml')
with open(experiment_configuration_file_path, "w") as f: 
  f.write(content)

## Defining the execution function

Next, the execution of a single experiment has to be defined. Note that this is a dummy example, which contains limited reasonable code. It is meant to show the core functionality of the PyExperimenter. 

The method is called with the parameters, i.e. `keyfields`, of a database entry. The results are meant to be processed to be written into the database, i.e. as `resultfields`. 

In [2]:
import random

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from py_experimenter.experimenter import PyExperimenter
from py_experimenter.result_processor import ResultProcessor

from time import sleep
from random import randint

def run_svm(parameters: dict, result_processor: ResultProcessor, custom_config: dict):
    sleep(randint(0,5))
    seed = parameters['seed']
    random.seed(seed)
    np.random.seed(seed)

    data = load_iris()

    X = data.data
    y = data.target

    # Create Support Vector Machine with parameters dependent on the kernel
    kernel = parameters['kernel']
    if kernel == 'linear':
        svc = SVC(kernel=parameters['kernel'])
    elif kernel == 'poly':
        svc = SVC(kernel=parameters['kernel'], gamma=parameters['gamma'], coef0=parameters['coef0'], degree=parameters['degree'])
    elif kernel == 'rbf':
        svc = SVC(kernel=parameters['kernel'], gamma=parameters['gamma'])

    svc = SVC()

    model = make_pipeline(StandardScaler(), svc)  

    if parameters['dataset'] != 'iris':
        raise ValueError("Example error")

    scores = cross_validate(model, X, y, 
        cv=parameters['cross_validation_splits'],
        scoring=('accuracy', 'f1_micro'),
        return_train_score=True
    )
    
    result_processor.process_results({
        'train_f1': np.mean(scores['train_f1_micro']),
        'train_accuracy': np.mean(scores['train_accuracy'])
    })

    result_processor.process_results({
        'test_f1': np.mean(scores['test_f1_micro']),
        'test_accuracy': np.mean(scores['test_accuracy'])})

## Executing PyExperimenter

The actual execution of the PyExperimenter is done in multiple steps. 

### Initialize PyExperimenter
The PyExperimenter is initialized with the previously created configuration file. Additionally, `PyExperimenter` is given a `name`, i.e. job id, which is especially useful for parallel executions of multiple experiments on HPC. 

In [3]:
from py_experimenter.experimenter import PyExperimenter

experimenter = PyExperimenter(experiment_configuration_file_path=experiment_configuration_file_path, name="SVM_experimenter_01")

2024-03-11 08:04:22,464  | py-experimenter - WARNING  | No values given for keyfield dataset
2024-03-11 08:04:22,465  | py-experimenter - WARNING  | No values given for keyfield cross_validation_splits
2024-03-11 08:04:22,466  | py-experimenter - WARNING  | No values given for keyfield seed
2024-03-11 08:04:22,466  | py-experimenter - WARNING  | No values given for keyfield kernel
2024-03-11 08:04:22,467  | py-experimenter - WARNING  | No values given for keyfield gamma
2024-03-11 08:04:22,468  | py-experimenter - WARNING  | No values given for keyfield degree
2024-03-11 08:04:22,468  | py-experimenter - WARNING  | No values given for keyfield coef0
2024-03-11 08:04:22,469  | py-experimenter - INFO     | Found 7 keyfields
2024-03-11 08:04:22,470  | py-experimenter - INFO     | Found 4 resultfields
2024-03-11 08:04:22,470  | py-experimenter - WARNING  | No logtables given
2024-03-11 08:04:22,471  | py-experimenter - WARNING  | No custom section defined in config
2024-03-11 08:04:22,471 

### Fill Table

The table is filled programmatically using the  `fill_table_from_combination()` method. We first generate the fixed parameter combinations for each kernel of the SVM in the first three lines.
* For the `rbf` kernel, we need to set values for the `gamma` parameter. The degree and `coef0` parameter are not present in this kernel, so we set these to `'nan'`.
* For the `poly` kernel, we need to set the `gamma`, the degree as well as the `coef0` parameter.
* For the `linear` kernel, we do not need to set any parameters, so all of them are set to `'nan'`.

Afterwards, we combine these with the seed, the dataset and the cross_validation_splits parameters, which are present for all experiment runs. Thus, these are not set unconditionally.

 

Note that the table can easily be obtained as `pandas.Dataframe` via `experimenter.get_table()`.

In [4]:
# Create parameter configurations for each kernel
combinations = [{'kernel': 'rbf', 'gamma': gamma, 'degree':None, 'coef0':None} for gamma in ['0.1','0.3']]
combinations += [{'kernel': 'poly', 'gamma': gamma, 'degree': degree, 'coef0': coef0} for gamma in ['0.1','0.3'] for degree in ['3','4'] for coef0 in ['0.0', '0.1']]
combinations += [{'kernel': 'linear','gamma': None, 'degree':None, 'coef0':None}]

# Fill experimenter
experimenter.fill_table_from_combination(parameters={'seed': ['1', '2', '3', '4', '5'], 
'dataset': ['iris'],
'cross_validation_splits': ['5'] },
fixed_parameter_combinations=combinations)

# showing database table
experimenter.get_table()

2024-03-11 08:04:22,555  | py-experimenter - INFO     | 55 rows successfully added to database. 0 rows were skipped.


,ID,dataset,cross_validation_splits,seed,kernel,gamma,degree,coef0,creation_date,status,start_date,name,machine,train_f1,train_accuracy,test_f1,test_accuracy,end_date,error
0,1,iris,5,1,rbf,0.1,None,None,2024-03-11 08:04:22,created,None,None,None,None,None,None,None,None,None
1,2,iris,5,1,rbf,0.3,None,None,2024-03-11 08:04:22,created,None,None,None,None,None,None,None,None,None
2,3,iris,5,1,poly,0.1,3,0.0,2024-03-11 08:04:22,created,None,None,None,None,None,None,None,None,None
3,4,iris,5,1,poly,0.1,3,0.1,2024-03-11 08:04:22,created,None,None,None,None,None,None,None,None,None
4,5,iris,5,1,poly,0.1,4,0.0,2024-03-11 08:04:22,created,None,None,None,None,None,None,None,None,None
5,6,iris,5,1,poly,0.1,4,0.1,2024-03-11 08:04:22,created,None,None,None,None,None,None,None,None,None
6,7,iris,5,1,poly,0.3,3,0.0,2024-03-11 08:04:22,created,None,None,None,None,None,None,None,None,None
7,8,iris,5,1,poly,0.3,3,0.1,2024-03-11 08:04:22,created,None,None,None,None,None,None,None,None,None
8,9,iris,5,1,poly,0.3,4,0.0,2024-03-11 08:04:22,created,None,None,None,None,None,None,None,None,None
9,10,iris,5,1,poly,0.3,4,0.1,2024-03-11 08:04:22,created,None,None,None,None,None,None,None,None,None


### Execute PyExperimenter
All experiments are executed one after the other by the same `PyExperimenter` due to `max_experiments=-1`. If just a single one or a predifined number of experiments should be executed, the `-1` has to be replaced by the according amount.
The first parameter, i.e. `run_svm`, relates to the actual method that should be executed with the given keyfields of the table. 

In [5]:
experimenter.execute(run_svm, max_experiments=-1)

# showing database table
experimenter.get_table() 

[codecarbon INFO @ 08:04:22] [setup] RAM Tracking...
[codecarbon INFO @ 08:04:22] [setup] GPU Tracking...
[codecarbon INFO @ 08:04:22] No GPU found.
[codecarbon INFO @ 08:04:22] [setup] CPU Tracking...
[codecarbon WARNING @ 08:04:22] No CPU tracking mode found. Falling back on CPU constant mode.


[codecarbon WARNING @ 08:04:23] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1260P but we don't know it. Please contact us.
[codecarbon INFO @ 08:04:23] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1260P
[codecarbon INFO @ 08:04:23] >>> Tracker's metadata:
[codecarbon INFO @ 08:04:23]   Platform system: Linux-5.15.146.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 08:04:23]   Python version: 3.9.0
[codecarbon INFO @ 08:04:23]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 08:04:23]   Available RAM : 15.475 GB
[codecarbon INFO @ 08:04:23]   CPU count: 16
[codecarbon INFO @ 08:04:23]   CPU model: 12th Gen Intel(R) Core(TM) i7-1260P
[codecarbon INFO @ 08:04:23]   GPU count: None
[codecarbon INFO @ 08:04:23]   GPU model: None
[codecarbon INFO @ 08:04:26] Energy consumed for RAM : 0.000000 kWh. RAM Power : 5.803128719329834 W
[codecarbon INFO @ 08:04:26] Energy consumed for all CPUs : 0.000001 kWh. Total CPU Power : 42.5 W
[codecarbon INF

,ID,dataset,cross_validation_splits,seed,kernel,gamma,degree,coef0,creation_date,status,start_date,name,machine,train_f1,train_accuracy,test_f1,test_accuracy,end_date,error
0,1,iris,5,1,rbf,0.1,None,None,2024-03-11 08:04:22,done,2024-03-11 08:04:22,SVM_experimenter_01,Worklaptop,0.975,0.975,0.966667,0.966667,2024-03-11 08:04:26,None
1,2,iris,5,1,rbf,0.3,None,None,2024-03-11 08:04:22,done,2024-03-11 08:04:27,SVM_experimenter_01,Worklaptop,0.975,0.975,0.966667,0.966667,2024-03-11 08:04:32,None
2,3,iris,5,1,poly,0.1,3,0.0,2024-03-11 08:04:22,done,2024-03-11 08:04:32,SVM_experimenter_01,Worklaptop,0.975,0.975,0.966667,0.966667,2024-03-11 08:04:37,None
3,4,iris,5,1,poly,0.1,3,0.1,2024-03-11 08:04:22,done,2024-03-11 08:04:37,SVM_experimenter_01,Worklaptop,0.975,0.975,0.966667,0.966667,2024-03-11 08:04:43,None
4,5,iris,5,1,poly,0.1,4,0.0,2024-03-11 08:04:22,done,2024-03-11 08:04:43,SVM_experimenter_01,Worklaptop,0.975,0.975,0.966667,0.966667,2024-03-11 08:04:48,None
5,6,iris,5,1,poly,0.1,4,0.1,2024-03-11 08:04:22,done,2024-03-11 08:04:48,SVM_experimenter_01,Worklaptop,0.975,0.975,0.966667,0.966667,2024-03-11 08:04:53,None
6,7,iris,5,1,poly,0.3,3,0.0,2024-03-11 08:04:22,done,2024-03-11 08:04:54,SVM_experimenter_01,Worklaptop,0.975,0.975,0.966667,0.966667,2024-03-11 08:04:59,None
7,8,iris,5,1,poly,0.3,3,0.1,2024-03-11 08:04:22,done,2024-03-11 08:04:59,SVM_experimenter_01,Worklaptop,0.975,0.975,0.966667,0.966667,2024-03-11 08:05:04,None
8,9,iris,5,1,poly,0.3,4,0.0,2024-03-11 08:04:22,done,2024-03-11 08:05:04,SVM_experimenter_01,Worklaptop,0.975,0.975,0.966667,0.966667,2024-03-11 08:05:10,None
9,10,iris,5,1,poly,0.3,4,0.1,2024-03-11 08:04:22,done,2024-03-11 08:05:10,SVM_experimenter_01,Worklaptop,0.975,0.975,0.966667,0.966667,2024-03-11 08:05:15,None


### CodeCarbon
Note that `CodeCarbon` is activated by default, collecting information about the carbon emissions of each experiment. Have a look at our [general usage example](https://tornede.github.io/py_experimenter/examples/example_general_usage.html) and the according [documentation of CodeCarbon fields](https://tornede.github.io/py_experimenter/usage.html#codecarbon-fields) for more information.